<a href="https://colab.research.google.com/github/garrettgcraig/colab-gee-test/blob/main/GC_CP270P_GEEDownload.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# =========================================================
# Before using this code, copy it to your own google drive.
# File > Save a copy in Drive
# =========================================================

In [2]:
# =========================================================
# Google Earth Engine - Amazonia Forest Cover Data
# This notebook:
# - Defines the Amazon basin boundary
# - Loads Hansen Global Forest Change data (tree cover, loss, gain)
# - Aligns everything to a template grid (same CRS, scale, rows/cols, extent)
# - Displays the layers
# - Exports Hansen forest data for the Amazon basin
# Requirements: Google Cloud Project ID, Google drive
# =========================================================

In [3]:
# Install GEE
!pip -q install geemap earthengine-api

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 11.1 MB/s eta 0:00:00


In [4]:
# Import packages
import ee
from datetime import datetime
import geemap
import os
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
# Connect GEE to your account (you need a GEE account linked to a project. Free for edu)
ee.Authenticate()
# Set your Cloud Project ID (Use your own project id)
PROJECT_ID = "gee-colab-test-478517"
# Initialize with your own project ID
ee.Initialize(project=PROJECT_ID)

In [6]:
### Helper Functions
# Function to align images to the template grid (same rows/cols & pixel origin)
def align_to_template(img, method='bilinear'):
    # resample for continuous data; use 'nearest' for categorical
    return (img
            .resample(method)
            .reproject(tpl_proj)    # force same CRS, transform, scale
            .clip(amazonia))

# Export function
def export_tif(img, name):
  path = f"{outdir}/{name}.tif"
  geemap.ee_export_image(
      ee_object=img,
      filename=path,
      region=amazonia.bounds(1, tpl_proj),  # bounds snapped to template
      scale=float(tpl_scale.getInfo()),
      crs=tpl_crs,
      file_per_band=False
  )
  print("Saved:", path)

#Export helper
outdir = "/content/drive/MyDrive/amazonia_hansen_data"  # output directory
os.makedirs(outdir, exist_ok=True)

In [17]:
# Set study area - Amazonia
# Using WWF Terrestrial Ecoregions to define Amazon biome
# Or use a simplified bounding box: [-80, -20, -44, 5]

# Option 1: Use WWF ecoregions for precise Amazon basin boundary
#ecoregions = ee.FeatureCollection('RESOLVE/ECOREGIONS/2017')
#amazonia = (ecoregions
#    .filter(ee.Filter.eq('BIOME_NAME', 'Tropical & Subtropical Moist Broadleaf Forests'))
#    .filter(ee.Filter.stringContains('ECO_NAME', 'Amazon'))
#    .geometry()
#)

# Option 2: Simple bounding box (uncomment to use instead)
amazonia = ee.Geometry.Rectangle([-80, -20, -44, 15])

In [18]:
# Set template grid - using Hansen's native resolution (30m)
# For large areas like Amazonia, you may want to use a coarser resolution
# Options: 30m (native), 100m, 250m, 1000m depending on your needs

# Get Hansen data to use as template
hansen = ee.Image("UMD/hansen/global_forest_change_2024_v1_12")
template = hansen.select("treecover2000").clip(amazonia)
tpl_proj = template.projection()
tpl_crs  = tpl_proj.crs()
tpl_scale = ee.Number(250)  # Use 250m for manageable file sizes (change to 30 for native resolution)

# Update projection with desired scale
tpl_proj = tpl_proj.atScale(tpl_scale)

In [19]:
### Load Hansen Global Forest Change data
# Hansen dataset includes multiple bands:
# - treecover2000: Tree canopy cover in year 2000 (0-100%)
# - loss: Forest loss during 2001-2024 (binary: 0 or 1)
# - gain: Forest gain during 2000-2012 (binary: 0 or 1)
# - lossyear: Year of forest loss (1-24 for years 2001-2024)
# - datamask: Data mask

hansen = ee.Image("UMD/hansen/global_forest_change_2024_v1_12")

# Select bands of interest
treecover2000 = hansen.select("treecover2000").toFloat()  # 0-100%
forest_loss = hansen.select("loss").toFloat()  # binary
forest_gain = hansen.select("gain").toFloat()  # binary
lossyear = hansen.select("lossyear").toFloat()  # 1-24

# Calculate net forest change (optional)
# Pixels with gain = 1, pixels with loss = -1
net_change = forest_gain.subtract(forest_loss)

In [20]:
# Visualization parameters for Hansen data
tc_vis = {"min": 0, "max": 100, "palette": ["#ffffcc", "#41ab5d", "#004529"]}  # Tree cover
loss_vis = {"min": 0, "max": 1, "palette": ["#ffffff", "#ff0000"]}  # Forest loss (red)
gain_vis = {"min": 0, "max": 1, "palette": ["#ffffff", "#0000ff"]}  # Forest gain (blue)
lossyear_vis = {"min": 1, "max": 24, "palette": ["#ffffb2", "#feb24c", "#f03b20", "#bd0026"]}  # Loss year

# Create interactive map centered on Amazonia
m = geemap.Map(center=[-8, -62], zoom=5)

# Add Hansen layers
m.addLayer(treecover2000.clip(amazonia), tc_vis, "Tree Cover 2000 (%)")
m.addLayer(forest_loss.clip(amazonia).updateMask(forest_loss), loss_vis, "Forest Loss 2001-2024", False)
m.addLayer(forest_gain.clip(amazonia).updateMask(forest_gain), gain_vis, "Forest Gain 2000-2012", False)
m.addLayer(lossyear.clip(amazonia).updateMask(lossyear), lossyear_vis, "Loss Year", False)
m.addLayer(amazonia, {"color": "black"}, "Amazonia boundary", False)

# Add colorbars
m.add_colorbar(tc_vis, label="Tree Cover 2000 (%)")
m.add_colorbar(lossyear_vis, label="Loss Year (1=2001, 24=2024)")

m  # Display map

Map(center=[-8, -62], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(chi…

In [11]:
# Export Hansen data for Amazonia
# NOTE: Amazonia is very large - exports may take significant time and storage
# Consider exporting at coarser resolution (e.g., 250m or 1km) or by country/subregion

# Export tree cover 2000
export_tif(align_to_template(treecover2000, 'bilinear'), "hansen_treecover2000_pct")

# Export forest loss (use 'nearest' for binary data)
export_tif(align_to_template(forest_loss, 'nearest'), "hansen_forest_loss_2001_2024")

# Export forest gain (use 'nearest' for binary data)
export_tif(align_to_template(forest_gain, 'nearest'), "hansen_forest_gain_2000_2012")

# Export loss year (use 'nearest' for categorical data)
export_tif(align_to_template(lossyear, 'nearest'), "hansen_lossyear")

print("\nAll exports complete! Check your Google Drive folder: amazonia_hansen_data")
print("\nNOTE: For very large regions like Amazonia, consider:")
print("- Using a coarser resolution (modify tpl_scale in cell 8)")
print("- Exporting by country or subregion")
print("- Using ee.batch.Export.image.toDrive() for background processing")

Generating URL ...
An error occurred while downloading.
Total request size (174998645 bytes) must be less than or equal to 50331648 bytes.
Saved: /content/drive/MyDrive/amazonia_hansen_data/hansen_treecover2000_pct.tif
Generating URL ...
An error occurred while downloading.
Image.resample: Invalid interpolation mode: nearest.
Saved: /content/drive/MyDrive/amazonia_hansen_data/hansen_forest_loss_2001_2024.tif
Generating URL ...
An error occurred while downloading.
Image.resample: Invalid interpolation mode: nearest.
Saved: /content/drive/MyDrive/amazonia_hansen_data/hansen_forest_gain_2000_2012.tif
Generating URL ...
An error occurred while downloading.
Image.resample: Invalid interpolation mode: nearest.
Saved: /content/drive/MyDrive/amazonia_hansen_data/hansen_lossyear.tif

All exports complete! Check your Google Drive folder: amazonia_hansen_data

NOTE: For very large regions like Amazonia, consider:
- Using a coarser resolution (modify tpl_scale in cell 8)
- Exporting by country or 